<a href="https://colab.research.google.com/github/hr2002/Generative-AI/blob/main/autoencoder_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras import models,layers
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras import datasets
(x_train , y_train), (x_test , y_test) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
def preprocess(imgs):
  imgs = imgs.astype("float32") / 255.0
  imgs = np.pad(imgs,((0,0),(2,2),(2,2)), constant_values=0.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs
x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [4]:
#ENCODER
encoder_input = layers.Input(shape = (32,32,1),name = "encoder_input")
x = layers.Conv2D(32,(3,3),strides = 2, activation='relu',padding = "same")(encoder_input)
x = layers.Conv2D(64,(3,3),strides = 2, activation='relu',padding = "same")(x)
x = layers.Conv2D(128,(3,3),strides = 2, activation='relu',padding = "same")(x)
shape_before_flattening = K.int_shape(x)[1:]
x = layers.Flatten()(x)
encoder_output = layers.Dense(2, name="encoder_output")(x)
encoder = models.Model(encoder_input, encoder_output)

In [5]:
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96770 (378.01 KB)
Trainable params: 96770 (378.

In [6]:
#decodeur

decoder_input = layers.Input(shape=(2,), name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(128,(3,3), strides=2,activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(64,(3,3), strides=2,activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(32,(3,3), strides=2,activation = 'relu', padding="same")(x)
decoder_output = layers.Conv2D(1,(3,3),strides = 1, activation="sigmoid",padding="same",name="decoder_output")(x)
decoder = models.Model(decoder_input, decoder_output)

In [7]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 2048)              6144      
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 32, 32, 32)        1846

In [8]:
autoencoder = models.Model(encoder_input, decoder(encoder_output))

In [9]:
#compile l'autoencodeur
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")


In [ ]:
autoencoder.fit(x_train,x_train,epochs=5,batch_size=100,shuffle=True,validation_data=(x_test,x_test),)

Epoch 1/5
600/600 [==============================] - 191s 310ms/step - loss: 0.2902 - val_loss: 0.2612
Epoch 2/5
600/600 [==============================] - 182s 303ms/step - loss: 0.2578 - val_loss: 0.2576
Epoch 3/5
600/600 [==============================] - 184s 307ms/step - loss: 0.2546 - val_loss: 0.2544
Epoch 4/5
 49/600 [=>............................] - ETA: 2:37 - loss: 0.2524

In [ ]:
example_images = x_test[:5000]
predictions = autoencoder.predict(example_images)

In [ ]:
fig, axes = plt.subplots(10, 3, figsize=(15, 2*10))

for i in range(10):
    # Plot original image
    axes[i, 0].imshow(example_images[i].reshape(32, 32), cmap='gray')
    axes[i, 0].set_title('Original')
    axes[i, 0].axis('off')

    # Plot reconstruction
    axes[i, 1].imshow(predictions[i].reshape(32, 32), cmap='gray')
    axes[i, 1].set_title('Reconstruction')
    axes[i, 1].axis('off')

    # Plot encoded representation (middle layer output)
    encoded_rep = encoder.predict(example_images[i][np.newaxis, :])
    axes[i, 2].imshow(encoded_rep.reshape(1, 2), cmap='gray')  # Assuming the encoded representation is 8x8
    axes[i, 2].set_title('Encoded Representation')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
embeddings = encoder.predict(example_images)
plt.figure(figsize=(8,8))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c="black", alpha=0.5, s=3)
plt.show()

In [ ]:
mins , maxs = np.min(embeddings, axis=0), np.max(embeddings, axis=0)
sample = np.random.uniform(mins, maxs, size=(18,2))
reconstructions = decoder.predict(sample)

In [ ]:
fig, axes = plt.subplots(10, 3, figsize=(15, 2*10))

for i in range(10):
    # Plot original image
    axes[i, 0].imshow(sample[i].reshape(1,2))
    axes[i, 0].set_title('Original')
    axes[i, 0].axis('off')

    # Plot reconstruction
    axes[i, 1].imshow(reconstructions[i].reshape(32, 32))
    axes[i, 1].set_title('Reconstruction')
    axes[i, 1].axis('off')

    # Plot encoded representation (middle layer output)
    encoded_rep = decoder.predict(sample[i][np.newaxis, :])
    axes[i, 2].imshow(encoded_rep.reshape(32, 32))  # Assuming the encoded representation is 8x8
    axes[i, 2].set_title('Encoded Representation')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()